<a href="https://colab.research.google.com/github/btripura/VQA_MODEL/blob/main/VQA_RESULT_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install datasets==1.17.0 nltk==3.5 pandas==1.3.5 Pillow scikit-learn==0.23.2 torch==1.12.0 transformers==4.15.0 dvc==2.9.3

In [ ]:
!pip install --pre allennlp-models

In [ ]:
import cv2
import numpy as np
import math
from google.colab import files
from IPython.display import Image
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
imdb="gdrive/MyDrive"
from allennlp_models.pretrained import load_predictor
predictor = load_predictor("vqa-vilbert")

In [ ]:
uploaded = files.upload()

In [ ]:
import os
from typing import Dict, List, Optional, Tuple
import numpy as np
from PIL import Image

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array,array_to_img

In [ ]:
from collections import deque
import cv2, numpy as np
import time
import math as mth
from PIL import Image
import random
import argparse
from hmac import trans_36
from shutil import move
from socket import CAN_BCM_RX_CHANGED
from cv2 import detail_AffineBasedEstimator
from keras.models import Sequential
from keras import initializers
from keras.initializers import normal, identity
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array


from IPython.display import Image,display
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import array_to_img
import PIL.Image
import io
from transformers import VisualBertForQuestionAnswering, BertTokenizerFast
import torch

In [ ]:

delta=10


#Image Dimensions
H=224
W=224

X = 0
Y = 0


st_x=0
e_x=W
st_y=0
e_y=H

xval=0
yval=0

In [ ]:
train_q=[]
with open('train_questions.txt','r') as fp:
  for line in fp:
    x=line[:-1]
    train_q.append(x)
#train_q = np.load('gdrive/MyDrive/train_ques_tokenizer.npy')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_q)
vocab_size = len(tokenizer.word_index) + 1
print('\n--- Converting questions to bags of words...')
train_X_seqs = tokenizer.texts_to_matrix(train_q)


--- Converting questions to bags of words...


In [ ]:
def load_and_process_image(image_path):
       im = PIL.Image.open(image_path)
       im=im.resize((224,224))
       im=np.array(im)
       return im

In [ ]:
test_ims=[]
test_ims.append(load_and_process_image('img4.jpg'))

In [ ]:
my_ques='What is color of bus'
test_q=[]
test_q.append(my_ques)

# We add one because the Keras Tokenizer reserves index 0 and never uses it.
test_X_seqs=tokenizer.texts_to_matrix(test_q)

In [ ]:
from PIL import Image
def transform(image,x_val,y_val):
    global X
    global Y
    X=x_val
    Y=y_val
    M=np.float32([[1,0,X],[0,1,Y]])
    output=cv2.warpAffine(image,M,(224,224))
    fdf=[X,Y]
    return cropped(output)

def cropped(image):
        start_x=st_x
        start_y=st_y
        end_x=e_x
        end_y=e_y
        values=[X,Y]
        if(X<0):
           end_x=W+X
        if(X>0):
           start_x=X
        if(Y<0):
           end_y=H+Y
        if(Y>0):
           start_y=Y
        crop=image[start_y:end_y,start_x:end_x]
        resized=cv2.resize(crop,(W,H),interpolation=cv2.INTER_NEAREST)
        return resized

In [ ]:
def predict(image,ques):
    URL='temp.jpg'
    preds = predictor.predict(URL, ques)
    best_prob, best_answer = max(zip(preds["probs"], preds["tokens"]), key=lambda x: x[0])
    return (best_answer,(best_prob/100))

In [ ]:
def step(state,ques,action):
    global xval
    global yval
    xval=X
    yval=Y
    if action==0:
        xval+=delta
    elif action==1:
        xval-=delta
    elif action==2:
        yval+=delta
    elif action==3:
        yval-=delta
    elif action==4:
      pass
    vad=[xval,yval]
    new_image=transform(state[0],xval,yval)
    image=array_to_img(new_image)
    image.save('temp.jpg')
    predict_ans,new_conf=predict(image,ques)
    new_image=img_to_array(new_image)
    return (new_image,predict_ans,new_conf)

In [ ]:
from tensorflow.keras.models import load_model
model=load_model('gdrive/MyDrive/model_final.h5')

In [ ]:
def get_action(state):
    q_values=model.predict([np.array([state[0]]),np.array([state[1]])]) #Exploitation
    max_Q = np.argmax(q_values[0])
    action = max_Q

    return action

In [ ]:
def reset():
  global X
  global Y
  global st_x
  global st_x
  global e_x
  global st_y
  global e_y
  global xval
  global yval
  xval=0
  yval=0
  X=0
  Y=0
  st_x=0
  e_x=224
  st_y=0
  e_y=224

In [ ]:
reset()
pred_answers=[]
pred_without_rein=[]
correct_answers=[]
answers=[]
confidence=[]
img=test_ims[0]
ques=test_X_seqs[0]
state=[img,ques]
for j in range(5):
    action=get_action(state)
    new_img,ans,conf=step(state,my_ques,action)
    answers.append(ans)
    confidence.append(conf)
    state=[new_img,ques]
pred_ans=answers[confidence.index(max(confidence))]
print('Question---',my_ques)
print('Answer-----',pred_ans)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred_vals,correct_vals):
  acc= accuracy_score(correct_vals, pred_vals)
  f1= f1_score(correct_vals, pred_vals, average='macro')
  return (acc,f1)

In [ ]:
pred_answers=[]
pred_without_rein=[]
for i in range(len(test_ims)):
  reset()
  answers=[]
  confidence=[]
  img=test_ims[i]
  ques=test_q[i]
  state=[img,test_X_seqs[i]]
  pred,con=predict(img,ques)
  pred_without_rein.append(pred)
  for j in range(steps):
    action=get_action(state)
    new_img,ans,conf=step(state,ques,action)
    answers.append(ans)
    confidence.append(conf)
    state=[new_img,test_X_seqs[i]]
  pred_ans=answers[confidence.index(max(confidence))]
  pred_answers.append(pred_ans)
  correct_answers.append(test_a[i])
accuracy_without_rein,f1_without_rein=compute_metrics(pred_without_rein,correct_answers)
accuracy,f1=compute_metrics(pred_answers,correct_answers)
print('Accuracy without Reinforcement Learning-----',accuracy_without_rein*100)
print('Accuracy with Reinforcement Learning-----',accuracy*100)

